In [82]:
import numpy as np
import scipy
import scipy.io
from scipy.spatial.transform import Rotation as R
import astropy.time
import datetime
import spiceypy as spice
import astropy.constants as const

In [9]:
earth_coord = np.array([-1.3965684e+08,57464451.,-7311051.4])
earth_vel = np.array([-11.633191,-26.864498,3.4187237])

sw_vel = np.array([-361.23,144.081,-12.45060])

xprime_gse = -earth_coord
xprime_gse = xprime_gse/np.linalg.norm(xprime_gse)

zprime_gse = np.cross(np.array([1,0,0]),xprime_gse)
zprime_gse = zprime_gse/np.linalg.norm(zprime_gse)
if zprime_gse[2] < 0:
    zprime_gse = -zprime_gse

yprime_gse = np.cross(zprime_gse, xprime_gse)
rot_matrix = np.row_stack((xprime_gse,yprime_gse,zprime_gse))
rotation = R.from_matrix(rot_matrix)

rotation.apply(sw_vel - earth_vel)

array([-388.65873034,   24.54586737,    5.83260756])

In [35]:
dateobs = datetime.datetime(year=2005,month=5,day=14,hour=0)
dateobs_jd = astropy.time.Time(dateobs).jd
dateobs_mjd = astropy.time.Time(dateobs).mjd
deg2rad = 180.0/np.pi
n = dateobs_jd - 2451545.0
L = 280.460 + 0.9856474*n
g = 357.528 + 0.9856003*n
Lambda = L + 1.915*np.sin(g*deg2rad) + 0.020*np.sin(2*g*deg2rad)
Omega = 73.6667 + 0.013958*(dateobs_mjd + 3243.75)/365.25
Omega_p = 75.76 + 1.397*n/36525.0
inc = 7.25
Theta = np.arctan(np.cos(inc)*np.tan(Lambda - Omega))





In [ ]:
Omega_p = 75.76 + 1.397*n/36525.0

In [39]:
v0_earth = 29.7859
r_matrix_HAE_HCI = R.from_euler("zy",[75.76*deg2rad,inc*deg2rad])
r_matrix_HCI_HAE = r_matrix_HAE_HCI.inv()
vel_earth_HAE =  v0_earth*np.array([np.cos(Lambda*deg2rad + np.pi/2),np.sin(Lambda*deg2rad + np.pi/2),0])
vel_earth_HCI = r_matrix_HAE_HCI.apply(vel_earth_HAE)
vel_earth_HCI

array([  7.17683512, -28.25823355,  -6.09631962])

In [75]:
r_matrix_HGI_HAE = R.from_euler("zy",[75.77*deg2rad,inc*deg2rad])
r_matrix_HGI_HAE.inv().apply([23.490618,-17.797121 ,0.00038074552])

array([23.2256371 , -9.89080026, 15.20817219])

In [65]:
vec_earth_HCI = np.array([-1.3999653e+08 ,56673694.,-7210420.1])
vec_earth_HCI = vec_earth_HCI/np.linalg.norm(vec_earth_HCI)
r_matrix_HGI_HCI = R.from_euler("z",[1.397*np.pi/180])
r_matrix_HGI_HCI.apply(vec_earth_HCI)

array([[-0.93473558,  0.3521298 , -0.04768641]])

In [64]:
vec_earth_HGI = np.array([-201.24,81.18,-10.33000])
vec_earth_HGI = vec_earth_HGI/np.linalg.norm(vec_earth_HGI)
vec_earth_HGI

array([-0.92633675,  0.37368325, -0.04755048])

In [79]:
coord_sav = scipy.io.readsav("../src/earth_coord_HAED.save",verbose=True)

--------------------------------------------------
Date: Tue Mar 15 11:52:22 2022
User: yjzhu
Host: Yingjies-MacBook-Pro.local
--------------------------------------------------
Format: 11
Architecture: x86_64
Operating System: darwin
IDL Version: 8.2
--------------------------------------------------
Successfully read 5 records of which:
 - 1 are of type VERSION
 - 1 are of type TIMESTAMP
 - 2 are of type VARIABLE
--------------------------------------------------
Available variables:
 - coord_j2000 [<class 'numpy.ndarray'>]
 - coord_050514 [<class 'numpy.ndarray'>]
--------------------------------------------------


In [94]:
r_j2000 = coord_sav["coord_j2000"][0:3]*1e3
v_j2000 = coord_sav["coord_j2000"][3:]*1e3

GM_earth = const.GM_earth.value
GM_sun = const.GM_sun.value

E = np.dot(v_j2000,v_j2000)/2 - (GM_earth + GM_sun)/np.linalg.norm(r_j2000)
h = np.linalg.norm(np.cross(r_j2000,v_j2000))

In [95]:
E

-443331792.3151136

In [96]:
h

4456247350402224.0

In [99]:
def get_vr_vphi_earth(coord_HCI,rsun=None):
    #It seems SWMF uses Rsun = 6.96E5 km
    if rsun is None:
        rsun = const.R_sun.value
    
    if type(coord_HCI) is list:
        coord_HCI = np.array(coord_HCI)

    # convert coord into SI units 
    x_earth = coord_HCI*rsun
    r_earth = np.linalg.norm(x_earth)

    GM_earth = const.GM_earth.value
    GM_sun = const.GM_sun.value

    # specific mechanical energy and specific angular momentum
    # calculated from the position and velocity of Earth on 
    # J2000 in HAE by SolarSoft get_sunspice_coord

    E = -443331792.3151136
    h = 4456247350402224.0

    vphi = h/r_earth
    vr = np.sqrt(2*E + 2*(GM_sun + GM_earth)/r_earth - np.square(vphi))

    return vr/1E3, vphi/1E3



In [101]:
get_vr_vphi_earth([-201.24,81.18,-10.33000])

(29.485050237288032, 0.4176996342921215)